<div align="center">

  <a href="https://ultralytics.com/yolo" target="_blank">
    <img width="1024", src="https://raw.githubusercontent.com/ultralytics/assets/main/yolov8/banner-yolov8.png"></a>

  [中文](https://docs.ultralytics.com/zh/) | [한국어](https://docs.ultralytics.com/ko/) | [日本語](https://docs.ultralytics.com/ja/) | [Русский](https://docs.ultralytics.com/ru/) | [Deutsch](https://docs.ultralytics.com/de/) | [Français](https://docs.ultralytics.com/fr/) | [Español](https://docs.ultralytics.com/es/) | [Português](https://docs.ultralytics.com/pt/) | [Türkçe](https://docs.ultralytics.com/tr/) | [Tiếng Việt](https://docs.ultralytics.com/vi/) | [العربية](https://docs.ultralytics.com/ar/)

  <a href="https://github.com/ultralytics/ultralytics/actions/workflows/ci.yaml"><img src="https://github.com/ultralytics/ultralytics/actions/workflows/ci.yaml/badge.svg" alt="Ultralytics CI"></a>
  <a href="https://console.paperspace.com/github/ultralytics/ultralytics"><img src="https://assets.paperspace.io/img/gradient-badge.svg" alt="Run on Gradient"/></a>
  <a href="https://colab.research.google.com/github/ultralytics/ultralytics/blob/main/examples/object_counting.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
  <a href="https://www.kaggle.com/models/ultralytics/yolo11"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>
  <a href="https://ultralytics.com/discord"><img alt="Discord" src="https://img.shields.io/discord/1089800235347353640?logo=discord&logoColor=white&label=Discord&color=blue"></a>

Welcome to the Ultralytics YOLO11 🚀 notebook! <a href="https://github.com/ultralytics/ultralytics">YOLO11</a> is the latest version of the YOLO (You Only Look Once) AI models developed by <a href="https://ultralytics.com">Ultralytics</a>. This notebook serves as the starting point for exploring the various resources available to help you get started with YOLO11 and understand its features and capabilities.

YOLO11 models are fast, accurate, and easy to use, making them ideal for various object detection and image segmentation tasks. They can be trained on large datasets and run on diverse hardware platforms, from CPUs to GPUs.

We hope that the resources in this notebook will help you get the most out of YOLO11. Please browse the YOLO11 <a href="https://docs.ultralytics.com/guides/object-counting/"> Object Counting Docs</a> for details, raise an issue on <a href="https://github.com/ultralytics/ultralytics">GitHub</a> for support, and join our <a href="https://ultralytics.com/discord">Discord</a> community for questions and discussions!

</div>

# Setup

Pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![Downloads](https://static.pepy.tech/badge/ultralytics)](https://www.pepy.tech/projects/ultralytics) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

In [1]:
%pip install ultralytics
import ultralytics

ultralytics.checks()

Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.1/112.6 GB disk)


# Object Counting using Ultralytics YOLO11 🚀

## What is Object Counting?

Object counting with [Ultralytics YOLO11](https://github.com/ultralytics/ultralytics/) involves accurate identification and counting of specific objects in videos and camera streams. YOLO11 excels in real-time applications, providing efficient and precise object counting for various scenarios like crowd analysis and surveillance, thanks to its state-of-the-art algorithms and deep learning capabilities.

## Advantages of Object Counting?

- **Resource Optimization:** Object counting facilitates efficient resource management by providing accurate counts, and optimizing resource allocation in applications like inventory management.
- **Enhanced Security:** Object counting enhances security and surveillance by accurately tracking and counting entities, aiding in proactive threat detection.
- **Informed Decision-Making:** Object counting offers valuable insights for decision-making, optimizing processes in retail, traffic management, and various other domains.

## Real World Applications

|                                                                           Logistics                                                                           |                                                                     Aquaculture                                                                     |
|:-------------------------------------------------------------------------------------------------------------------------------------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------:|
| ![Conveyor Belt Packets Counting Using Ultralytics YOLO11](https://github.com/RizwanMunawar/ultralytics/assets/62513924/70e2d106-510c-4c6c-a57a-d34a765aa757) | ![Fish Counting in Sea using Ultralytics YOLO11](https://github.com/RizwanMunawar/ultralytics/assets/62513924/c60d047b-3837-435f-8d29-bb9fc95d2191) |
|                                                    Conveyor Belt Packets Counting Using Ultralytics YOLO11                                                    |                                                    Fish Counting in Sea using Ultralytics YOLO11                                                    |


In [13]:
import cv2
from ultralytics import solutions

# Open the video file
cap = cv2.VideoCapture("/content/vid1.mp4")
assert cap.isOpened(), "Error reading video file"

# Get video properties: width, height, and frames per second (fps)
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define the region of interest (line) horizontally in the middle of the frame
line_y = h // 2  # Midpoint of the frame height (horizontal line)
line_points = [(0, line_y), (w, line_y)]  # Line across the whole width of the frame

# Initialize counters for objects moving top to bottom (in_count) and bottom to top (out_count)
in_count = 0
out_count = 0

# Initialize the video writer to save the output video
video_writer = cv2.VideoWriter("object_counting_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Initialize the Object Counter with visualization options and other parameters
counter = solutions.ObjectCounter(
    show=True,  # Display the image during processing
    region=line_points,  # Region of interest points (horizontal line)
    model="yolo11n.pt",  # Ultralytics YOLO11 model file
    line_width=2,  # Thickness of the lines and bounding boxes
)

# Store previous y-coordinates of objects to track direction (top to bottom or bottom to top)
previous_y_coords = {}

# Process video frames in a loop
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Use the Object Counter to count objects in the frame and get the annotated image
    im0 = counter.count(im0)

    # Debug: Print the structure of the detected objects
    print("Detected Objects:", counter.names)

    # Access the list of detected objects from counter.names
    detected_objects = counter.names  # Let's print to inspect the format

    # Debugging: Print each object to see its structure
    for obj in detected_objects:
        print(f"Object Details: {obj}")

    # Assuming 'counter' returns bounding box data (i.e., a list of dictionaries or objects),
    # you will need to access bounding boxes like:
    # - 'bbox' or a similar key containing the coordinates

    for obj in detected_objects:
        # Assuming 'obj' is now a dictionary or object with bounding box information:
        if isinstance(obj, dict) and 'bbox' in obj:  # Check if bbox is available in the object
            bbox = obj['bbox']  # Example: {'bbox': [x_min, y_min, x_max, y_max]}

            # Calculate the centroid of the bounding box (middle point of the object)
            x_center = (bbox[0] + bbox[2]) / 2
            y_center = (bbox[1] + bbox[3]) / 2

            # Check if the object is crossing the line from top to bottom or vice versa
            if y_center > line_y:  # Object is moving downward (top to bottom)
                if obj['id'] not in previous_y_coords:  # If it's a new object
                    in_count += 1
            elif y_center < line_y:  # Object is moving upward (bottom to top)
                if obj['id'] not in previous_y_coords:  # If it's a new object
                    out_count += 1

            # Update the previous y-coordinate of the object
            previous_y_coords[obj['id']] = y_center

    # Annotate the frame with in_count and out_count
    cv2.putText(im0, f"In: {in_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(im0, f"Out: {out_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write the annotated frame to the output video
    video_writer.write(im0)

# Release the video capture and writer objects
cap.release()
video_writer.release()

# Close all OpenCV windows
cv2.destroyAllWindows()


Streaming output truncated to the last 5000 lines.
Object Details: 37
Object Details: 38
Object Details: 39
Object Details: 40
Object Details: 41
Object Details: 42
Object Details: 43
Object Details: 44
Object Details: 45
Object Details: 46
Object Details: 47
Object Details: 48
Object Details: 49
Object Details: 50
Object Details: 51
Object Details: 52
Object Details: 53
Object Details: 54
Object Details: 55
Object Details: 56
Object Details: 57
Object Details: 58
Object Details: 59
Object Details: 60
Object Details: 61
Object Details: 62
Object Details: 63
Object Details: 64
Object Details: 65
Object Details: 66
Object Details: 67
Object Details: 68
Object Details: 69
Object Details: 70
Object Details: 71
Object Details: 72
Object Details: 73
Object Details: 74
Object Details: 75
Object Details: 76
Object Details: 77
Object Details: 78
Object Details: 79

0: 384x640 19 persons, 11.9ms
Speed: 5.5ms preprocess, 11.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Dete

# Additional Resources

## Community Support

For more information on counting objects with Ultralytics, you can explore the comprehensive [Ultralytics Object Counting Docs](https://docs.ultralytics.com/guides/object-counting/). This guide covers everything from basic concepts to advanced techniques, ensuring you get the most out of counting and visualization.

## Ultralytics ⚡ Resources

At Ultralytics, we are committed to providing cutting-edge AI solutions. Here are some key resources to learn more about our company and get involved with our community:

- [Ultralytics HUB](https://ultralytics.com/hub): Simplify your AI projects with Ultralytics HUB, our no-code tool for effortless YOLO training and deployment.
- [Ultralytics Licensing](https://ultralytics.com/license): Review our licensing terms to understand how you can use our software in your projects.
- [About Us](https://ultralytics.com/about): Discover our mission, vision, and the story behind Ultralytics.
- [Join Our Team](https://ultralytics.com/work): Explore career opportunities and join our team of talented professionals.

## YOLO11 🚀 Resources

YOLO11 is the latest evolution in the YOLO series, offering state-of-the-art performance in object detection and image segmentation. Here are some essential resources to help you get started with YOLO11:

- [GitHub](https://github.com/ultralytics/ultralytics): Access the YOLO11 repository on GitHub, where you can find the source code, contribute to the project, and report issues.
- [Docs](https://docs.ultralytics.com/): Explore the official documentation for YOLO11, including installation guides, tutorials, and detailed API references.
- [Discord](https://ultralytics.com/discord): Join our Discord community to connect with other users, share your projects, and get help from the Ultralytics team.

These resources are designed to help you leverage the full potential of Ultralytics' offerings and YOLO11. Whether you're a beginner or an experienced developer, you'll find the information and support you need to succeed.